In [140]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import dateutil.parser
import matplotlib.pyplot as plt
# --------------------------- #
#        Data Preprocessing   #
# --------------------------- #

def load_and_preprocess_site_data(site_path, window_size=32, horizon=16, min_date=None, max_date=None):
    """
    Loads and preprocesses time series data for a given site.

    Args:
    - site_path (str): Path to the site CSV file.
    - window_size (int): Number of past time steps for input.
    - horizon (int): Number of future steps to predict.

    Returns:
    - train_loader, val_loader, test_loader: DataLoaders for training, validation, and testing.
    """
    df = pd.read_csv(site_path)
    
    # Convert date column to datetime if it exists
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])

        # Filter data between min_date and max_date
        if min_date:
            min_date = dateutil.parser.parse(min_date) if isinstance(min_date, str) else min_date
            df = df[df['date'] >= min_date]

        if max_date:
            max_date = dateutil.parser.parse(max_date) if isinstance(max_date, str) else max_date
            df = df[df['date'] <= max_date]

        # Drop the date column after filtering
        df.drop(columns=['date'], inplace=True)
        
    
    # Perform an 80-20 split based on time order
    train_size = int(0.8 * len(df))
    train_df = df.iloc[:train_size]  # 80% for training & validation
    test_df = df.iloc[train_size:]   # 20% for final testing (future unseen data)

    # Split train_df further into Train (80%) and Validation (20%)
    val_size = int(0.2 * len(train_df))  # 16% of full dataset
    train_df, val_df = train_df.iloc[:-val_size], train_df.iloc[-val_size:]

    print(f"Train size: {len(train_df)} | Validation size: {len(val_df)} | Test size: {len(test_df)}")

    # Standardize each separately to prevent data leakage
    train_mean, train_std = train_df.mean(), train_df.std()
    
    train_df = (train_df - train_mean) / (train_std + 1e-8)
    val_df = (val_df - train_mean) / (train_std + 1e-8)  # Normalize validation using train stats
    test_df = (test_df - train_mean) / (train_std + 1e-8)  # Normalize test using train stats

    # Convert DataFrame to NumPy arrays for LSTM
    X_train, y_train = df_to_X_y(train_df,window_size, horizon)
    X_val, y_val = df_to_X_y(val_df, window_size, horizon)
    X_test, y_test = df_to_X_y(test_df, window_size, horizon)

    # Convert to PyTorch tensors
    train_data = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
    val_data = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
    test_data = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

    # Create DataLoaders
    batch_size = 32
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)
    val_loader = DataLoader(val_data, shuffle=False, batch_size=batch_size, drop_last=True)
    test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=True)

    return train_loader, val_loader, test_loader


def df_to_X_y(df, window_size=32, horizon=16):
    """
    Converts a DataFrame into supervised learning format for LSTM.

    Args:
    - df (pd.DataFrame): DataFrame containing time series.
    - window_size (int): Past window size.
    - horizon (int): Number of future steps.

    Returns:
    - X (np.array): Features.
    - y (np.array): Targets.
    """
    if "OT" not in df.columns:
        raise ValueError("The target column 'OT' is missing from the DataFrame.")

    df_as_np = df.to_numpy()
    target_col_idx = df.columns.get_loc("OT")  # Get the index of 'OT' column

    X, y = [], []

    for i in range(len(df_as_np) - window_size - horizon + 1):
        X.append(df_as_np[i:i + window_size])  # Past 'window_size' rows as features
        y.append(df_as_np[i + window_size:i + window_size + horizon, target_col_idx])  # Next 'horizon' steps for target

    return np.array(X), np.array(y)



# --------------------------- #
#          LSTM Model         #
# --------------------------- #

class LSTMModel(nn.Module):
    def __init__(self, num_features, time_window, output_window, num_labels, num_layers=2, hidden_size=16, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.output_window = output_window
        self.num_labels = num_labels

        self.lstm = nn.LSTM(num_features,
                            hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout)
        
        self.fc = nn.Linear(hidden_size, num_labels * output_window)

    def forward(self, x):
        B, T, D = x.shape  
        x_, _ = self.lstm(x)
        last_hidden = x_[:, -1, :]  # Take last time step's hidden state
        x_ = self.fc(last_hidden)
        x_ = x_.reshape(B, self.output_window, self.num_labels)
        return x_


# --------------------------- #
#      Training Function      #
# --------------------------- #

def train_model(model, train_loader, val_loader, num_epochs=30, lr=1e-3, wd=1e-5):
    """
    Trains an LSTM model with validation.

    Args:
    - model (nn.Module): The LSTM model.
    - train_loader (DataLoader): DataLoader for training.
    - val_loader (DataLoader): DataLoader for validation.
    - num_epochs (int): Number of training epochs.
    - lr (float): Learning rate.

    Returns:
    - None
    """
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=wd)
    device = "cuda"

    model.to(device)
    
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        # print(f"\nEPOCH {epoch+1}")
        
        # -------------------- Training -------------------- #
        model.train()
        running_train_loss = 0.0
        
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            output = model(batch_x)
            loss = criterion(output, batch_y.unsqueeze(-1))  # Ensure correct shape
            running_train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_train_loss = running_train_loss / len(train_loader)
        avg_train_loss = running_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # -------------------- Validation -------------------- #
        model.eval()
        running_val_loss = 0.0

        with torch.no_grad():  # Disable gradient calculations
            for batch_x, batch_y in val_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                output = model(batch_x)
                loss = criterion(output, batch_y.unsqueeze(-1))  # Ensure correct shape
                running_val_loss += loss.item()

        avg_val_loss = running_val_loss / len(val_loader)
        avg_val_loss = running_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)


        # Print losses for the epoch
        print(f"Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

    print("Training complete!")
     # -------------------- Plot Loss Curves -------------------- #

    # plt.figure(figsize=(8, 5))
    # plt.plot(range(1, num_epochs + 1), train_losses, label="Training Loss", marker='o')
    # plt.plot(range(1, num_epochs + 1), val_losses, label="Validation Loss", marker='s')
    
    # plt.xlabel("Epoch")
    # plt.ylabel("Loss")
    # plt.title("Training and Validation Loss Curves")
    # plt.legend()
    # plt.grid(True)
    # plt.show()

    return train_losses, val_losses



# --------------------------- #
#      Evaluation Function    #
# --------------------------- #

def evaluate_model(model, test_loader):
    """
    Evaluates the LSTM model on test data.

    Args:
    - model (nn.Module): Trained LSTM model.
    - test_loader (DataLoader): DataLoader for testing.

    Returns:
    - mean_mae (float): Mean Absolute Error.
    """
    model.eval()
    criterion = nn.L1Loss()
    mae_list = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")
            predictions = model(batch_x)
            mae = criterion(predictions, batch_y.unsqueeze(-1)).item()
            mae_list.append(mae)

    return np.mean(mae_list)



In [141]:
site_directory = "../processed_ds/stores_data/"
site_files = [os.path.join(site_directory, f, f"{f}.csv") for f in os.listdir(site_directory) if os.path.isdir(os.path.join(site_directory, f))]

In [ ]:
# --------------------------- #
#    Run Experiments for All Sites #
# --------------------------- #

# Directory containing all site data
# site_directory = "../processed_ds/air_quality_cluster/"
# min_date = "2014-09-01"
# max_date = "2014-11-12 19:00"
# num_features = 11
# site_directory = "../processed_ds/solar/"
# min_date = "2006-09-01"
# max_date = "2006-09-08 4:50"
# num_features = 10
# site_directory = "../processed_ds/crypto-data/"
# min_date = "2018-04-01"
# max_date = "2018-06-15"
# num_features = 5
site_directory = "../processed_ds/stores_data/"
min_date = "2013-01-16"
max_date = "2015-07-31"
num_features = 7
site_files = [os.path.join(site_directory, f, f"{f}.csv") for f in os.listdir(site_directory) if os.path.isdir(os.path.join(site_directory, f))]

all_mae = []

for site_path in site_files:
    print(f"\nProcessing Site: {site_path}")

    # Load data Crypto target_col = 3
    train_loader, val_loader, test_loader = load_and_preprocess_site_data(site_path,min_date=min_date, max_date=max_date)

    # Define LSTM Model
    model = LSTMModel(num_features=num_features, time_window=32, output_window=16, num_labels=1, num_layers=2, hidden_size=128, dropout=0.1)

    # Train the model
    train_model(model, train_loader, val_loader ,num_epochs=30, lr=1e-3, wd=1e-3)

    # Evaluate the model
    mae = evaluate_model(model, test_loader)
    print(f"MAE for {site_path}: {mae:.4f}")
    all_mae.append(mae)

# Compute average MAE across all sites
avg_mae = np.mean(all_mae)
print(f"\nAverage MAE Across All Sites: {avg_mae:.4f}")


Processing Site: ../processed_ds/stores_data/store-631/store-631.csv
Train size: 593 | Validation size: 148 | Test size: 186
Train Loss: 0.9824 | Validation Loss: 0.7408
Train Loss: 0.9603 | Validation Loss: 0.7109
Train Loss: 0.8804 | Validation Loss: 0.5920
Train Loss: 0.7531 | Validation Loss: 0.5344
Train Loss: 0.6534 | Validation Loss: 0.4000
Train Loss: 0.5077 | Validation Loss: 0.2973
Train Loss: 0.4216 | Validation Loss: 0.2184
Train Loss: 0.3476 | Validation Loss: 0.1925
Train Loss: 0.3100 | Validation Loss: 0.1664
Train Loss: 0.3000 | Validation Loss: 0.1682
Train Loss: 0.2918 | Validation Loss: 0.1757
Train Loss: 0.2939 | Validation Loss: 0.1676
Train Loss: 0.2913 | Validation Loss: 0.1810
Train Loss: 0.2919 | Validation Loss: 0.1691
Train Loss: 0.2882 | Validation Loss: 0.1704
Train Loss: 0.2859 | Validation Loss: 0.1623
Train Loss: 0.2879 | Validation Loss: 0.1585
Train Loss: 0.2837 | Validation Loss: 0.1851
Train Loss: 0.2792 | Validation Loss: 0.1812
Train Loss: 0.2791 